In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
# 5 = Class 0, 7 = Class 1
x57_train = x_train[(y_train == 5) | (y_train == 7),]
y57_train = y_train[(y_train == 5) | (y_train == 7)]
y57_train = np.where(y57_train == 5, 0, 1).reshape((-1,1))

x57_test = x_test[(y_test == 5) | (y_test == 7),]
y57_test = y_test[(y_test == 5) | (y_test == 7)]
y57_test = np.where(y57_test == 5, 0, 1).reshape((-1,1))

In [4]:
x57_train = np.expand_dims(x57_train, axis = 3)
x57_train = np.repeat(x57_train,3,axis = 3)
x57_train = tf.image.resize(x57_train, [32,32])/255

x57_test = np.expand_dims(x57_test, axis = 3)
x57_test = np.repeat(x57_test,3,axis = 3)
x57_test = tf.image.resize(x57_test, [32,32])/255

In [7]:
x57_train[0].shape

TensorShape([32, 32, 3])

In [5]:
base_model = VGG16(input_shape = (x57_train[0].shape[0],x57_train[0].shape[1],3),
                   include_top = False, weights = 'imagenet')

In [6]:
model_name = 'mnist57_oracle'

inputs = tf.keras.Input(shape=(x57_train[0].shape[0],x57_train[0].shape[1],3))
x = base_model(inputs, training=False)
x = Flatten()(x)
outputs = Dense(1, activation = 'sigmoid')(x)
model = tf.keras.Model(inputs, outputs, name = model_name)
model.summary()

Model: "mnist57_oracle"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 14,715,201
Non-trainable params: 0
_________________________________________________________________


In [7]:
train_gen = ImageDataGenerator(
    rotation_range=20,
    shear_range=10, validation_split = 0.2)
train_gen.fit(x57_train)

In [8]:
lr = 0.0001
batchsize = 50
path ="./oracles/mnist57.h5"
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_accuracy', patience = 3 , restore_best_weights=True)
# checkpoint = ModelCheckpoint(path, monitor = 'val_accuracy', save_best_only=True)
callbacks = [earlystop]

In [9]:
model.fit(train_gen.flow(x57_train, y57_train, batch_size = batchsize, subset = 'training'),
          validation_data=train_gen.flow(x57_train, y57_train, batch_size = batchsize, subset = 'validation'),
          steps_per_epoch=len(x57_train)*0.8/batchsize,
          epochs = 100, 
          callbacks = callbacks)
model.save(path, save_format = 'h5')

Epoch 1/100
186/186 [==============================] - 14s 48ms/step - loss: 0.0244 - accuracy: 0.9910 - val_loss: 0.0324 - val_accuracy: 0.9932
Epoch 2/100
186/186 [==============================] - 7s 40ms/step - loss: 0.0050 - accuracy: 0.9986 - val_loss: 2.6084e-04 - val_accuracy: 1.0000
Epoch 3/100
186/186 [==============================] - 8s 41ms/step - loss: 0.0063 - accuracy: 0.9982 - val_loss: 0.0012 - val_accuracy: 0.9996
Epoch 4/100
186/186 [==============================] - 7s 39ms/step - loss: 0.0038 - accuracy: 0.9990 - val_loss: 1.3444e-04 - val_accuracy: 1.0000
Epoch 5/100
186/186 [==============================] - 7s 39ms/step - loss: 8.5737e-04 - accuracy: 0.9996 - val_loss: 0.0025 - val_accuracy: 0.9987


In [10]:
model = load_model(path)
model.summary()

Model: "mnist57_oracle"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 14,715,201
Non-trainable params: 0
_________________________________________________________________


In [11]:
preds = model.predict(x57_test)
preds = np.squeeze(np.round(preds, 0).astype('int'))

In [12]:
np.mean(preds == np.squeeze(y57_test))

1.0